In [1]:
import pandas as pd 
import numpy as np
pd.set_option('display.max_columns', 500)
pd.set_option('display.max_rows', 500)
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from sklearn.linear_model import Ridge
from sklearn.ensemble import RandomForestRegressor
from sklearn.model_selection import cross_val_predict
from sklearn.model_selection import cross_val_score
from sklearn.svm import LinearSVR
import xgboost as xgb
from sklearn.preprocessing import StandardScaler
MAX_TOWERS = 6
from catboost import CatBoostRegressor
from sklearn.preprocessing import LabelEncoder
np.random.seed(17)
import random
random.seed(17)
from sklearn.model_selection import KFold
from sklearn.metrics import mean_absolute_error

/anaconda2/envs/env/lib/python3.6/site-packages/sklearn/ensemble/weight_boosting.py:29: DeprecationWarning: numpy.core.umath_tests is an internal NumPy module and should not be imported. It will be removed in a future NumPy release.
  from numpy.core.umath_tests import inner1d


In [2]:
def get_train():
    train_main = pd.read_csv("../data/task1/train_1.7.csv", encoding="cp1251")
    train_aux_coords = pd.read_csv("../data/task1_additional/coords_train_1.1.csv", encoding="cp1251")
    train_aux_frac = pd.read_csv("../data/task1_additional/frac_train_1.csv", encoding="cp1251")
    train_aux_gdis = pd.read_csv("../data/task1_additional/gdis_train1.2.csv", encoding="cp1251")
    
    
    train_frac_main = pd.merge(train_main, train_aux_frac,how="left", left_on="Скважина", right_on="Скважина")
    all_recs = pd.merge(train_frac_main, train_aux_gdis,how="left", left_on="Скважина", right_on="Скважина")
    print(all_recs.shape)
    return all_recs

def get_test():
    test_main = pd.read_csv("../data/task1/test_1.9.csv", encoding="cp1251")
    test_aux_coords = pd.read_csv("../data/task1_additional/coords_train_1.1.csv", encoding="cp1251")
    test_aux_frac = pd.read_csv("../data/task1_additional/frac_test_1.csv", encoding="cp1251")
    test_aux_gdis = pd.read_csv("../data/task1_additional/gdis_test1.2.csv", encoding="cp1251")
    
    
    test_frac_main = pd.merge(test_main, test_aux_frac,how="left", left_on="Скважина", right_on="Скважина")
    all_recs = pd.merge(test_frac_main, test_aux_gdis,how="left", left_on="Скважина", right_on="Скважина")
    print(all_recs.shape)
    return all_recs

In [73]:
#sort by converted date and group
def convert_and_sort(df):
    df["Дата"] =  df["Дата"].apply(pd.to_datetime)
    return df.sort_values(by=["Скважина", "Дата"])

def get_non_useful(df):
    non_useful_columns = []
    for c in df.columns:
        null_columns = df[df[c].isnull()]
        if len(null_columns)== len(df):
            non_useful_columns.append(c)
    return non_useful_columns

def drop_non_useful(train, test):
    non_useful = set(get_non_useful(train)) |set(get_non_useful(test))
    print("%s dropped"% non_useful)
    return train.drop(list(non_useful), axis=1), test.drop(list(non_useful), axis=1)

def get_float(v):
    v = str(v)
    if v != "NaN":
        new = v.replace(",",".")
        return float(new)
    return v

def get_target(df, column="Нефть, т"):
    target = df[column]
    print("%s dropped"% column)
    return df.drop([column], axis=1), target.apply(get_float)

#drop non present columns in test
def drop_not_present(train, test):
    absent_columns = list(set(train.columns) - set(test.columns))
    print("%s dropped"% absent_columns)
    return train.drop(absent_columns, axis=1), test
    
def show_uniq_test_train(train, test):
    #check all values that have zero ans nan only
    for c in sorted(train.columns):
        un = train[c].unique()
        if len(un)<100:
            tun = test[c].unique()
            print("%s ;train: %s; test:%s"%(c, un, tun))

In [74]:
show_uniq_test_train(get_train(), get_test())

(5735, 147)
(319, 138)
I X/X ;train: [14.  17.8  0.   nan 11.  11.5 22.  12.  16.5 24.  13. ]; test:[ nan  0.  11.  17.8 32. ]
JD ;train: [ 0.45               nan  0.55        0.43        0.51        0.46
  0.54        0.47        0.53        0.57        0.59        0.544
  0.23        0.32        0.32192551  0.37        0.33        0.31
  0.29        0.742       0.27        0.475349    0.5613433   0.45097399
  0.56        0.41208407  0.53957552  0.68        0.66        0.73
  0.8558197   0.77857057 80.          0.63        0.79        0.66222741
  0.5         0.81        0.78016436  0.86        0.74        0.41
  0.7         0.77486049  0.77535941  0.60964027  0.48986028  0.25706972
  0.35175216  0.62339058  0.7147887   0.71        0.47934847  0.43858438
  0.27518276  0.63532807  0.45493487  0.62        0.8         1.35642087
  0.61        0.27419876  0.33766811  0.34637102  0.6356302   0.21716332
  0.2641767   0.2208527   0.36233129  0.69        0.49124835  0.44
  0.68982446  0.77430

In [4]:
def common_data_pipeline(train, test):
    print(train.shape)
    print(test.shape)
    
    y = None
    train=convert_and_sort(train)
    train, test = drop_non_useful(train, test)
    #remove target from train
    train, y = get_target(train)
    train, test = drop_not_present(train, test)
    print(train.shape)
    print(test.shape)
    
    return train, test, y

In [82]:
def get_existed(columns, df):
    return list(set(columns)&set(df.columns))

def split_continious_date_categorical_text(df):
    group_id = ["Скважина"]
    text = ["Причина простоя"]
    categorical = ["Тип испытания",
                   "Тип скважины",
                   "Неустановившийся режим",
                   "ГТМ",
                   "Метод",
                   "Характер работы",
                   "Состояние",
                   "Пласт МЭР", 
                   "Способ эксплуатации", 
                   "Тип насоса", 
                   "Состояние на конец месяца", 
                   "Номер бригады", 
                   "Фонтан через насос", 
                   "Нерентабельная",
                   "Назначение по проекту",
                   "Группа фонда",
                   "Тип дополнительного оборудования",
                   "Марка ПЭД",
                   "Тип ГЗУ",
                   "ДНС",
                   "КНС",
                   #useless potentially
                   "Диаметр плунжера",
                   "Природный газ, м3",
                   "Конденсат, т",
                   "Длина хода плунжера ШГН",
                   "Коэффициент подачи насоса",
                   "Дебит конденсата",
                   "Вязкость воды в пластовых условиях",
                   "Газ из газовой шапки, м3",
                   "Число качаний ШГН",
                   "Коэффициент сепарации",
                   "SKIN",
                   "КН закрепленный",
                   # radically different
                   "Время в работе",
                   "Радиус контура питания",
                   "Время в накоплении",
                   "Время накопления",
                   "Агент закачки",
                   # text converted
                   "Мероприятия",
                   "Проппант",
                   "Куст",
                   "Состояние на конец месяца",
                   "Причина простоя.1",
                   ]
    dates = ["Дата", 
             "Дата ГРП",
             "Время до псевдоуст-ся режима", 
             "Дата запуска после КРС", 
             "Дата пуска", 
             "Дата останова",
             "Дата ввода в эксплуатацию"]
    
    continious = list(set(df.columns) - set(dates) - set(categorical) - set(text) - set(group_id))
    return (df[group_id],df[continious], df[get_existed(dates,df)], df[get_existed(categorical, df)],
            df[get_existed(text, df)])

In [6]:
def get_object_columns(df):
    objects = []
    for c in df.columns:
        if df[c].dtype != pd.np.float:
            objects.append(c)
    return objects

def convert_locale_to_float(df):
    loc_float = get_object_columns(df)
    converted = df.copy()
    for c in loc_float:
        converted.loc[:,c] = df[c].apply(get_float)
    return converted
        
def fill_with_mean(train, test):
    means=train.mean()
    norm_train = train.fillna(means)
    norm_test = test.fillna(means)
    return norm_train, norm_test

# now we have clear non-normalized data, let's normalize first
def normalize(train, test):
    scaler = StandardScaler()
    norm_train = pd.DataFrame(scaler.fit_transform(train), columns=train.columns, index = train.index)
    norm_test = pd.DataFrame(scaler.transform(test), columns=test.columns, index = test.index)
    return norm_train, norm_test

In [7]:
def cont_transform_pipeline(train, test):
    train_f = convert_locale_to_float(train)
    test_f = convert_locale_to_float(test)
    train_cont, test_cont = fill_with_mean(train_f, test_f)
    train_cont, test_cont = normalize(train_cont, test_cont)
    print(train_cont.isnull().values.any() or test_cont.isnull().values.any())
    
    print(train_cont.shape)
    print(test_cont.shape)
    return train_cont, test_cont

In [8]:
def transform_cats_to_labels(train_cat, test_cat):
    transformed_df = train_cat.copy()
    trans_test = test_cat.copy()
    for c in train_cat.columns:
        encoder = LabelEncoder()
        column_train = train_cat[c].astype(str)
        column_test = test_cat[c].astype(str)
        combined = pd.concat([column_train, column_test])
        encoder.fit(combined)
        transformed_df[c] = encoder.transform(column_train).reshape(-1,1)
        trans_test[c] = encoder.transform(column_test).reshape(-1,1)
    return transformed_df, trans_test

In [9]:
def cat_transform_pipeline(train, test):
    train_cat, test_cat = transform_cats_to_labels(train, test)
    print(train_cat.shape)
    print(test_cat.shape)
    return train_cat, test_cat

In [94]:
def text_trasnsform_pipeline(train, test):
    if "Причина простоя" in train.columns:
        train_txt = train["Причина простоя"].str.lower()
        test_txt = test["Причина простоя"].str.lower()
        return train_txt, test_txt
    return train, test

In [65]:
def transform_dates_into_order(dates, group):
    grouped = pd.concat([dates, group], axis=1)
    idx = []
    orders = []
    for name,group in grouped.groupby(["Скважина"]):
        index = group.index
        for i in range(len(index)):
            idx.append(index[i])
            orders.append(i)
    ord_index = pd.Index(idx)
    ordered_fr = pd.Series(orders, index=ord_index, dtype="int32",name="order")
    return ordered_fr

def dates_transform_pipeline(train, test, train_group, test_group):
    train_cont = train.drop(["Дата"], axis =1)
    train_cat = train["Дата"].apply(pd.to_datetime)
    train_cat = transform_dates_into_order(train_cat, train_group)
    test_cont = test.drop(["Дата"], axis =1)
    test_cat = test["Дата"].apply(pd.to_datetime)
    test_cat = pd.Series(0, index=test_cat.index, dtype="int32", name="order")
    return train_cont, test_cont, train_cat, test_cat

In [10]:
def clean_non_targeted(train_array, y_train):
    clean_array = []
    train_array.append(y_train)
    #clear nans in target
    indexes_to_delete = y_train[y_train.isnull()].index
    for df in train_array:
        item = df.drop(index=indexes_to_delete)
        clean_array.append(item)
        print(item.shape)
    return clean_array

In [11]:
def get_preds_for_cats(train, test, y):
    cb_regressor = CatBoostRegressor(logging_level="Silent")
    train_catboost_preds = cross_val_predict(cb_regressor, train, y)
    cb_regressor.fit(train,y=y)
    test_catboost_preds = pd.Series(cb_regressor.predict(test), index=test.index)
    return train_catboost_preds, test_catboost_preds

In [12]:
def get_cont_ensemble():
    ridge = Ridge()
    rtree = RandomForestRegressor(n_jobs=-1, n_estimators=50)
    svr = LinearSVR()
    return [ridge, rtree, svr]

def get_cont_ensemble_names():
    return ["ridge", "rtree", "svr"]

In [13]:
def get_meta_train_preds(X, y, train_mixture, mix_cols):
    predicts = []
    for cl in get_cont_ensemble():
        predicts.append(cross_val_predict(cl, X,y, n_jobs=-1))
    predicts.append(train_mixture)
    return pd.DataFrame(np.vstack(predicts).transpose(), index=y.index, columns=get_cont_ensemble_names()+mix_cols)

def get_meta_test_predict(X_train, y_train, X_test, test_mixture, mix_cols):
    test_predicts = []
    for cl in get_cont_ensemble():
        print(cross_val_score(cl, X_train, y_train, n_jobs=-1, scoring="neg_mean_absolute_error"))
        cl.fit(X_train, y_train)
        pr = cl.predict(X_test)
        test_predicts.append(pr)
    test_predicts.append(test_mixture)
    return  pd.DataFrame(np.vstack(test_predicts).transpose(), index=X_test.index, columns=get_cont_ensemble_names()+mix_cols)

def get_stacked_ensemble_predict(X_meta, y, X_test):
    regressor = xgb.XGBRegressor()
    regressor.fit(X_meta, y)
    return pd.DataFrame(regressor.predict(X_test), index=X_test.index)

In [14]:
def get_n_item_index(group_size, df, group):
    new_df = pd.concat([df, group], axis = 1)
    index = []
    group = new_df.groupby(["Скважина"])
    for name, group in group:
        if len(group)<group_size:
            continue
        for start in range(len(group.index) - group_size):
            gr =group.index[start:start+group_size]
            index.append(gr)
    return index

def get_timed_ds(meta_size, df, group, y):
    if meta_size >= 1:
        meta_indexes = get_n_item_index(meta_size, df, group)
        first_value_idx = []
        timed_ds = df.copy()
        metas = []
        columns = []
        for i in range(meta_size):
            columns.append("meta%s"%str(i))
        for a in meta_indexes:
            first_value_idx.append(a[0])
            metas.append(list(y.loc[a]))
        metas_df = pd.DataFrame.from_records(metas, index=first_value_idx, columns=columns)
        return pd.concat([timed_ds.loc[first_value_idx], metas_df], axis=1)
    elif meta_size == 0:
        return df
    
def get_n_tower_predictions(n, train, y, test, train_group, train_mix, test_mix, mix_col):
    X_meta_train = get_meta_train_preds(train, y, train_mixture=train_mix, mix_cols=mix_col)
    X_meta_test = get_meta_test_predict(train, y, test, test_mixture=test_mix, mix_cols=mix_col)
    test_predictions= []
    for i in range(0,n):
        train_timed_ds = get_timed_ds(i,X_meta_train, train_group, y)
        y_timed = y.loc[train_timed_ds.index]
        test_predict = get_stacked_ensemble_predict(train_timed_ds, y_timed,X_meta_test)
        test_predictions.append(test_predict)
        X_meta_test["meta%s"%i]=test_predict
    return pd.concat(test_predictions,axis=1)

In [99]:
def create_submission(fname, df):
    final_pred = pd.Series(df)
    final_pred.to_csv(fname,header=["_VAL_"],index_label=["_ID_"])

In [90]:
get_clean_data(get_train(), get_test())

(5735, 147)
(319, 138)
(5735, 147)
(319, 138)
{'Агент закачки', 'Станок-качалка', 'Фирма ГРП', 'Тип газосепаратора', 'Примечание'} dropped
Нефть, т dropped
['ТП(ИДН) Дебит жидкости скорр-ый', 'ТП(ГРП) Дебит жидкости скорр-ый', 'ТП(ГРП) Дебит жидкости', 'ГП - Общий прирост Qн', 'Дебит жидкости', 'ТП(ИДН) Дебит жидкости', 'Жидкость, м3', 'Нефть, м3'] dropped
(5735, 133)
(319, 133)
False
(5735, 89)
(319, 89)
(5735, 43)
(319, 43)
(4764, 89)
(4764, 1)
(4764, 43)
(4764,)


(            КВЧ  Коэффициент продуктивности        Азимут  Попутный газ, м3  \
 2724   0.000000                1.782976e-15  3.968614e-15          0.768743   
 2725   0.000000                1.782976e-15  3.968614e-15          0.768743   
 2726   0.000000                1.782976e-15  3.968614e-15         -0.811722   
 2727   0.000000                1.782976e-15  3.968614e-15         -0.811722   
 2728   0.000000                1.782976e-15  3.968614e-15         -0.811722   
 2729   0.000000                1.782976e-15  3.968614e-15         -0.811722   
 2730   0.000000                1.782976e-15  3.968614e-15         -0.811722   
 2731   0.000000                1.782976e-15  3.968614e-15         -0.811722   
 2732   0.000000                1.782976e-15  3.968614e-15         -0.811722   
 2733   0.000000                1.782976e-15  3.968614e-15         -0.811722   
 2734   0.000000                1.782976e-15  3.968614e-15         -0.811722   
 2735   0.000000                1.782976

In [79]:
def get_clean_data(train, test):
    train, test, y_train  = common_data_pipeline(train, test)
    train_group, train_cont, train_dat, train_cat, train_text = split_continious_date_categorical_text(train)
    test_group, test_cont, test_dat, test_cat, test_text = split_continious_date_categorical_text(test)
    train_dat_cont, test_dat_cont, train_dat_cat, test_dat_cat = dates_transform_pipeline(
        train_dat, test_dat, train_group, test_group)
    train_text, test_text = text_trasnsform_pipeline(train_text, test_text)
    train_cont = pd.concat([train_cont, train_dat_cont], axis = 1)
    train_cat = pd.concat([train_cat, train_dat_cat, train_text], axis = 1)
    test_cont = pd.concat([test_cont, test_dat_cont], axis = 1)
    test_cat = pd.concat([test_cat, test_dat_cat, test_text], axis = 1)
    train_cont, test_cont = cont_transform_pipeline(train_cont, test_cont)
    train_cat, test_cat = cat_transform_pipeline(train_cat, test_cat)
    train_cont, train_group, train_cat, y_train = clean_non_targeted([train_cont, train_group, train_cat], y_train)
    train_cat_preds, test_cat_preds = get_preds_for_cats(train_cat, test_cat, y_train)
    return train_cont, y_train, test_cont, train_group, train_cat_preds, test_cat_preds

In [17]:
def get_prediction(train, test, constant = 701.4750):
    train_cont, y_train, test_cont, train_group, train_cat_preds, test_cat_preds = get_clean_data(train,test)
    time_serie_pred = get_n_tower_predictions(
        6, train_cont, y_train,test_cont, train_group, train_cat_preds, test_cat_preds, ["catboost"]
    ).values.reshape(-1,1)
    time_serie_pred = np.squeeze(time_serie_pred)
    print(constant - np.mean(time_serie_pred))
    final_pred = time_serie_pred + (constant - np.mean(time_serie_pred))
    return final_pred

In [96]:
preds = get_prediction(get_train(), get_test())

(5735, 147)
(319, 138)
(5735, 147)
(319, 138)
{'Агент закачки', 'Станок-качалка', 'Фирма ГРП', 'Тип газосепаратора', 'Примечание'} dropped
Нефть, т dropped
['ТП(ИДН) Дебит жидкости скорр-ый', 'ТП(ГРП) Дебит жидкости скорр-ый', 'ТП(ГРП) Дебит жидкости', 'ГП - Общий прирост Qн', 'Дебит жидкости', 'ТП(ИДН) Дебит жидкости', 'Жидкость, м3', 'Нефть, м3'] dropped
(5735, 133)
(319, 133)
False
(5735, 89)
(319, 89)
(5735, 43)
(319, 43)
(4764, 89)
(4764, 1)
(4764, 43)
(4764,)
[-123.82927846 -129.99477131 -136.30924315]
[-49.83267683 -52.59621348 -61.17370302]
[ -90.64580542 -121.43579498 -110.71701249]
457.1884704589844


In [100]:
create_submission("all_data_submit.csv", preds)

In [103]:
!cat all_data_submit.csv

_ID_,_VAL_
0,459.73947
1,459.55997
2,459.50934
3,459.19238
4,457.37564
5,457.41077
6,745.7455
7,746.0824
8,748.8236
9,753.8505
10,743.8397
11,743.436
12,868.62646
13,861.20135
14,863.2027
15,867.9221
16,868.7803
17,869.0791
18,571.18445
19,567.9663
20,568.8611
21,568.792
22,570.6321
23,574.4501
24,459.73947
25,459.55997
26,459.52142
27,459.27542
28,457.37564
29,457.41077
30,630.11523
31,630.21075
32,630.5147
33,630.06616
34,630.4033
35,630.2381
36,833.6625
37,836.3733
38,836.8884
39,832.07605
40,831.9833
41,832.5474
42,1159.1467
43,1158.4775
44,1159.4983
45,1156.6293
46,1158.5767
47,1159.9116
48,1500.8676
49,1514.035
50,1509.1915
51,1500.7588
52,1505.4811
53,1495.6107
54,459.73947
55,459.55997
56,459.50934
57,459.19238
58,457.37564
59,457.41077
60,459.73947
61,459.55997
62,459.52142
63,459.19238
64,457.37564
65,457.41077
66,459.73947
67,459.55997
68,459.50934
69,459.19238
70,457.37564
71,457.41077
72,459.73947
73,459.55997
74,459.50934
75,459.19238
76,457.37564
77,457.41077
78,791.6207

In [1]:
def get_cleaned_test(test, size=6):
    test, y_test = get_target(test)
    test, y_test = clean_non_targeted([test], y_test)
    X_idx = []
    y_idx = []
    group = test.groupby(["Скважина"])
    for name, group in group:
        if len(group)!= size:
            continue
        X_idx.append(group.index[0])
        y_idx.extend(group.index)
    if X_idx and y_idx:
        test = test.loc[X_idx]
        y_test = y_test[y_idx]
        constant = np.mean(y_test.values)
        return test, y_test, constant
    else:
        return None, None, None

In [2]:
def cross_validate(train, n_folds = 4):
    errors = []
    k_fold = KFold(n_splits=n_folds)
    for tr_ix, test_ix in k_fold.split(train):
        print("new fold started")
        train_cv = train.iloc[tr_ix]
        test_cv = train.iloc[test_ix]
        test_cv, y_test, constant = get_cleaned_test(test_cv)
        test_preds = get_prediction(train_cv, test_cv, constant=constant)
        error=mean_absolute_error(y_true=y_test, y_pred=test_preds)
        print("fold error:%s" % error)
        errors.append(error)
    print("overall error: %s" % np.mean(errors))
    print(errors)

In [3]:
%%time
cross_validate(get_train())

NameError: name 'get_train' is not defined

In [ ]:
299.720573012412
[289.9811313784219, 335.9357574203388, 126.81710257918746, 446.1483006716997]
300.25068988386926
[290.65788832253554, 336.1340891623984, 127.1327437987151, 447.07803825182793]

# eda part

In [70]:
text = get_train()[["Причина простоя",
            "Куст",
            "Состояние на конец месяца",
            "Причина простоя.1",
            "Мероприятия",
            "Проппант"]]

(5735, 147)


In [34]:
dates.dtypes

Дата                             object
Дата ГРП                        float64
Время до псевдоуст-ся режима    float64
Дата запуска после КРС          float64
Дата пуска                      float64
Дата останова                   float64
Дата ввода в эксплуатацию       float64
dtype: object